In [54]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt

In [55]:
# Load images 
img1 = cv2.imread('../src/ransac1.jpg', cv2.IMREAD_GRAYSCALE) 
img2 = cv2.imread('../src/ransac2.jpg', cv2.IMREAD_GRAYSCALE) 

# Step 1: Feature Detection and Description 
detector = cv2.ORB_create() 
kp1, des1 = detector.detectAndCompute(img1, None) 
kp2, des2 = detector.detectAndCompute(img2, None) 
# Compute correspondences matrix
correspondences = np.zeros((des1.shape[0], des2.shape[0])) 
for i in range(des1.shape[0]): 
    for j in range(des2.shape[0]): 
        diff = np.sum((des1[i]-des2[j])**2) 
        correspondences[i,j] = np.sum((des1[i]-des2[j])**2) 

# Normalize correspondences matrix between 0 and 1
correspondences = (correspondences - np.min(correspondences)) / (np.max(correspondences) - np.min(correspondences)) 
correspondences = np.where(correspondences < .15, 1, 0)

# Draw correspondences
matches = []
for i in range(des1.shape[0]):
    for j in range(des2.shape[0]):
        if correspondences[i, j] == 1:
            matches.append(cv2.DMatch(i, j, 1))

# Draw matches
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [56]:
def line_two_point(p1, p2):
    direction_vector = [(b-a) for a,b in zip(p1,p2)]
    refrence_point   = p1
    return refrence_point, direction_vector
def distance(point, reference_point, direction_vector):
    # Convert lists to numpy arrays for easier computation
    point = np.array(point)
    reference_point = np.array(reference_point)
    direction_vector = np.array(direction_vector)
    
    # Calculate vector from reference point to the given point
    vector_to_point = point - reference_point
    
    # Calculate projection of the vector onto the line
    projection = np.dot(vector_to_point, direction_vector) / np.dot(direction_vector, direction_vector) * direction_vector
    
    # Calculate vector perpendicular to the line
    perpendicular_vector = vector_to_point - projection
    
    # Calculate the distance (magnitude of perpendicular vector)
    distance = np.linalg.norm(perpendicular_vector)
    
    return distance

ref, dir = line_two_point([1,0], [3,0])
distance([2, 1], ref, dir)

1.0

In [57]:
threshold = .5
def dist_score(p1, p2, corr_i, corr_j):
    ref, dir = line_two_point(p1, p2)
    dist = []
    for i in range(len(corr_i)):
        dist.append(distance(des1[corr_i[i]], ref, dir))
        dist.append(distance(des2[corr_j[i]], ref, dir))
    dist = np.array(dist)
    dist = (dist - np.min(dist)) / (np.max(dist) - np.min(dist))
    score = np.count_nonzero(dist < threshold)
    return score, dist

def makeRowCol2(vec):
    for i in range(len(vec)):
        vec[i] = 2 if vec[i] == 1 else 0
    return vec

def edit_correspond(dist, corr_i, corr_j):
    for i in range(len(corr_i)):
        if dist[i  ] > threshold: correspondences[corr_i[i],:] = makeRowCol2(correspondences[corr_i[i],:])
        if dist[i+1] > threshold: correspondences[:,corr_j[i]] = makeRowCol2(correspondences[:,corr_j[i]])
    
best_dist = 0
scores = np.zeros_like(correspondences)
corr_i, corr_j = np.where(correspondences == 1)
best_score = 0
for i in range(len(corr_i)):
    scores[corr_i[i], corr_j[i]], dst = dist_score(des1[corr_i[i]], des2[corr_j[i]], corr_i, corr_j)
    if best_score < scores[corr_i[i], corr_j[i]]: best_dist = dst

edit_correspond(best_dist, corr_i, corr_j)

# Draw correspondences
matches = []
for i in range(des1.shape[0]):
    for j in range(des2.shape[0]):
        if correspondences[i, j] == 1:
            matches.append(cv2.DMatch(i, j, 1))

# Draw matches
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

/tmp/ipykernel_6262/2820365774.py:2: RuntimeWarning: overflow encountered in scalar subtract
  direction_vector = [(b-a) for a,b in zip(p1,p2)]
/tmp/ipykernel_6262/2820365774.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  projection = np.dot(vector_to_point, direction_vector) / np.dot(direction_vector, direction_vector) * direction_vector
/tmp/ipykernel_6262/2820365774.py:15: RuntimeWarning: invalid value encountered in multiply
  projection = np.dot(vector_to_point, direction_vector) / np.dot(direction_vector, direction_vector) * direction_vector
/tmp/ipykernel_6262/2820365774.py:15: RuntimeWarning: invalid value encountered in scalar divide
  projection = np.dot(vector_to_point, direction_vector) / np.dot(direction_vector, direction_vector) * direction_vector
